# Regularized Linear Regression

BUSMGT 7247

## Load libraries and data

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
from pandas.plotting import scatter_matrix

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import (
    LinearRegression, Ridge, Lasso, ElasticNet
)
from sklearn.metrics import mean_squared_error, r2_score

import matplotlib as mpl
import matplotlib.pyplot as plt

# Make this notebook's output stable across runs
np.random.seed(100)

# Do not use scientific notation in pandas tables
pd.options.display.float_format = '{:,.4f}'.format

# Plot formatting
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

### Data Description

When retailers close stores, they usually conduct going-out-of-business (GOB) sales.

In the data set, each row represents a store's going out of business sale.

| Attribute             | Definition                                                                |
| --------------------- | ------------------------------------------------------------------------- |
| StoreType             | Store's retail segment (bookstores or household goods)                    |
| LiquidationRevenue    | Revenue from the GOB sale                                                 |
| InventoryAtTicket     | Retail value of inventory held at the start of the GOB sale               |
| LastYearRevenue       | Store's revenue over the GOB sale dates during the prior year             |                      
| MedianHouseholdIncome | Median household income in the store's ZIP code (from U.S. Census Bureau) |
| MedianAge             | Median age in the store's ZIP code (from U.S. Census Bureau)              |


In [2]:
df = pd.read_csv('data/closings.csv')
df.head()

,StoreType,LiquidationRevenue,InventoryAtTicket,LastYearRevenue,MedianHouseholdIncome,MedianAge
0,Bookstore,"1,214,776.2000","2,195,069.8000","465,237.3000",51290,37.8000
1,Bookstore,"1,811,896.9000","3,152,301.9000","897,704.5000",46702,33.4000
2,Bookstore,"1,157,614.7000","2,229,837.5000","477,804.8000",86804,41.7000
3,Bookstore,"2,037,136.2000","3,857,466.8000","1,036,097.7000",83544,40.3000
4,Bookstore,"1,326,821.1000","2,345,263.8000","612,793.5000",23508,31.9000


## Model

In [3]:
# Transform features
df['Bookstore'] = df['StoreType'].apply(lambda x: 1 if x == 'Bookstore' else 0)
df['BookstoreXInventoryAtTicket'] = df['Bookstore'] * df['InventoryAtTicket']

# Add square and cube terms
numvars = ['InventoryAtTicket', 'LastYearRevenue',
          'MedianHouseholdIncome', 'MedianAge']
for v in numvars:
    df[v + 'Sq'] = df[v]**2
    df[v + 'Cu'] = df[v]**3
    
# Fit a model that incorporates all variables
y = df['LiquidationRevenue']

feature_names = list(df.columns[2:])
X = df[feature_names]

# Scale data
scaler = StandardScaler()
X_scale = scaler.fit_transform(X)

# Split data into training and test sets
(X_train, X_test,
 y_train, y_test) = train_test_split(X_scale, y, test_size=0.75)

In [4]:
# Fit a linear regresion
linreg = LinearRegression()
linreg.fit(X_train, y_train)
y_pred = linreg.predict(X_test)

In [5]:
# Retain coefficients and performance
df_regs = pd.DataFrame(columns=['Intercept'] + feature_names + ['Test R2'])
df_regs.loc['Linear'] = [linreg.intercept_] + list(linreg.coef_) + [r2_score(y_test, y_pred)]

In [6]:
# Set regularization parameter for ridge, lasso, and elasticnet
alpha=2
l1_ratio=0.5

### Ridge Regression

In [7]:
ridge = Ridge(alpha=alpha)
ridge.fit(X_train, y_train)
y_pred = ridge.predict(X_test)

df_regs.loc['Ridge'] = [ridge.intercept_] + list(ridge.coef_) + [r2_score(y_test, y_pred)]

### LASSO Regression

In [8]:
lasso = Lasso(alpha=alpha, max_iter=100000)
lasso.fit(X_train, y_train)
y_pred = lasso.predict(X_test)

df_regs.loc['Lasso'] = [lasso.intercept_] + list(lasso.coef_) + [r2_score(y_test, y_pred)]

### Elastic Net Regression

In [9]:
elastic = ElasticNet(alpha=alpha, l1_ratio=l1_ratio)
elastic.fit(X_train, y_train)
y_pred = elastic.predict(X_test)

df_regs.loc['ElasticNet'] = [elastic.intercept_] + list(elastic.coef_) + [r2_score(y_test, y_pred)]

### Compare Regularization Methods

In [10]:
df_regs

,Intercept,InventoryAtTicket,LastYearRevenue,MedianHouseholdIncome,MedianAge,Bookstore,BookstoreXInventoryAtTicket,InventoryAtTicketSq,InventoryAtTicketCu,LastYearRevenueSq,LastYearRevenueCu,MedianHouseholdIncomeSq,MedianHouseholdIncomeCu,MedianAgeSq,MedianAgeCu,Test R2
Linear,"1,743,761.2840","35,348.0193","-246,953.5007","58,268.4570","469,147.7064","14,585.7644","-44,139.9834","926,612.9399","-817,907.2647","1,472,339.8287","-949,232.6769","130,154.0345","-211,064.5441","-1,132,144.4842","648,109.2204",0.7625
Ridge,"1,764,206.5118","179,118.0645","330,080.2278","109,552.8269","-26,683.2002","-84,362.7372","29,420.9039","48,720.5776","-58,982.1445","160,216.2384","-92,277.3346","-14,341.0543","-101,472.8931","-19,833.1823","-11,893.5960",0.9175
Lasso,"1,743,993.2892","45,379.9223","-241,790.3816","60,757.1020","450,276.9896","13,279.3897","-42,947.4332","897,062.3850","-797,849.6432","1,462,975.9094","-944,056.4457","124,501.8914","-207,537.0434","-1,091,159.6461","624,791.8015",0.7656
ElasticNet,"1,773,525.0702","84,469.7632","103,577.7075","12,258.2045","-8,922.1994","-79,634.1117","-58,469.6230","72,085.2462","58,621.5331","83,557.2852","56,460.6165","1,617.0178","-5,645.7604","-9,728.1085","-10,390.7233",0.8845
